In [ ]:
# !sudo pip3 install --upgrade pip keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn optuna scikit-image  optkeras

In [137]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile
import pickle
import pandas as pd

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

#Properties
props = Properties()
props["dagman.retry"] = "2"
props["pegasus.transfer.arguments"] = "-m 1"
props.write()

#Replica Catalog
rc = ReplicaCatalog()
input_files = glob('*.jpg')
input_files.sort()
in_files=[]

checkpoint_file = "checkpoint1.csv"


if not os.path.isfile(checkpoint_file):
    df = pd.DataFrame(list())
    df.to_csv(checkpoint_file)
        

for file in input_files:
    in_files.append(File(file))
    rc.add_replica("local", File(file), str(Path(".").resolve() / file))  
rc.add_replica("local", File(checkpoint_file), str(Path(".").resolve() / file)) 
rc.write()


#Transformation
pre_process_resize = Transformation( "preprocess1.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/preprocess1.py",
        is_stageable=True)

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Augmentation.py",
        is_stageable=True)

data_split  = Transformation( "Data_Split.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Data_Split.py",
        is_stageable=True)

vgg_model  = Transformation( "VGG_model.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/VGG_model.py",
        is_stageable=True)

hpo =  Transformation( "hpo_checkpointing.py",
        site = "local",
        pfn=str(Path(".").resolve() /"hpo_checkpointing.py"),
        is_stageable=True)
                    
tc = TransformationCatalog()\
    .add_transformations(pre_process_resize,pre_process_augment,data_split,vgg_model,hpo)\
    .write()

#Workflow
wf = Workflow("Cats_and_Dogs", infer_dependencies=True)


resized_images = File('resized_images.txt')
all_files = [File("resized_{}".format(f.lfn)) for f in in_files]
labels = File('labels.txt')

job_preprocess1 = Job(pre_process_resize)\
                    .add_inputs(*in_files)\
                    .add_outputs(*all_files,resized_images,labels) 

aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

    
job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(*all_files,labels)\
                    .add_outputs(aug_images_txt,aug_labels_txt,*augmented_files)

training_data = File('training.pkl')
testing_data = File('testing.pkl')
val_data = File('validation.pkl')

job_data_split = Job(data_split)\
                    .add_inputs(*augmented_files,labels)\
                    .add_outputs(training_data,testing_data,val_data)

model = File('model.h5')

job_vgg_model = Job(vgg_model)\
                    .add_inputs(*augmented_files,training_data,testing_data,val_data)\
                    .add_outputs(model)

job_hpo = Job(hpo)\
                    .add_checkpoint(File(checkpoint_file), stage_out=True)\
                    .add_inputs(model,*augmented_files,training_data,testing_data,val_data)\
                    .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=100)

wf.add_jobs(job_preprocess1,job_preprocess2,job_data_split,job_vgg_model,job_hpo)                                    

In [138]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.09.24 18:45:06.333 UTC:
2020.09.24 18:45:06.338 UTC:   -----------------------------------------------------------------------
2020.09.24 18:45:06.344 UTC:   File for submitting this DAG to HTCondor           : Cats_and_Dogs-0.dag.condor.sub
2020.09.24 18:45:06.349 UTC:   Log of DAGMan debugging messages                 : Cats_and_Dogs-0.dag.dagman.out
2020.09.24 18:45:06.355 UTC:   Log of 

[--------------------------------------------------]   0.0% ..Running (Completed: 0, Queued: 0, Running: 0, Failed: 0)--------------------------------------------------]   0.0% ..Running (Completed: 0, Queued: 0, Running: 2, Failed: 0)--------------------------------------------------]   0.0% ..Running (Completed: 0, Queued: 0, Running: 0, Failed: 0)####----------------------------------------------]   7.7% ..Running (Completed: 2, Queued: 6, Running: 0, Failed: 0)####----------------------------------------------]   7.7% ..Running (Completed: 2, Queued: 0, Running: 6, Failed: 0)####----------------------------------------------]   7.7% ..Running (Completed: 2, Queued: 0, Running: 6, Failed: 0)####----------------------------------------------]   7.7% ..Running (Completed: 2, Queued: 0, Running: 1, Failed: 0)#############-------------------------------------]  26.9% ..Running (Completed: 7, Queued: 0, Running: 0, Failed: 0)###############-----------------------------------]  30.8% ..Ru


####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/CatsAndDogs/scitech/pegasus/Cats_and_Dogs/run0027
Total jobs         :     26 (100.00%)
# jobs succeeded   :     26 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workf

In [139]:
import csv

with open("wf-output/checkpoint1.csv", encoding="utf8") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=",")

    for row in csvreader:
        print(": ".join(row))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [140]:
with open("wf-output/checkpoint1.csv", 'r', encoding='ISO-8859-1', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=",")
    for row in csvreader:
        print(', '.join(row))

Error: line contains NULL byte

In [126]:
import codecs
csvReader = csv.reader(codecs.open("wf-output/checkpoint.csv", 'rU', 'ISO-8859-1'))

In [127]:
for row in csvReader:
        print(', '.join(row))

Error: line contains NULL byte